# MovieLens 1m Collaborative Genre Tagging
**To do**:
  - try out [this implementation](https://www.onceupondata.com/2019/02/10/nn-collaborative-filtering/) of baseline features. 
  - create object classes for models
  - implement TF 2.0 data classes
  - [paperswithcode link](https://paperswithcode.com/sota/collaborative-filtering-on-movielens-100k)
  - [ML 100k state of the art paper](https://arxiv.org/pdf/1706.02263v2.pdf) (RMSE=0.905): details their evaluation method
  
[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/James-Leslie/deep-collaborative-filtering/blob/master/tf-movielens10m.ipynb)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os

import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Add, Dense, Concatenate, Dropout, LeakyReLU
from tensorflow.keras.models import Model

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

%matplotlib inline

## Load rating data

In [2]:
# path = 'https://raw.githubusercontent.com/James-Leslie/deep-collaborative-filtering/master/data/ml-100k/'  # access from anywhere
# path = 'data/ml-100k/'  # if the files are local
path = 'data/ml-10M100K/'  # ML-1m file

all_files = glob.glob(os.path.join(path, "ratings*.csv"))
df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

In [3]:
df.head()

,userId,movieId,rating
0,69587,1005,2.0
1,47904,193,4.0
2,26906,3097,3.5
3,31241,559,1.0
4,69402,2541,4.0


In [4]:
df.shape

(10000054, 3)

In [5]:
print('Number of users:', df.userId.nunique())
print('Number of items:', df.movieId.nunique())
print("Min item rating:", df.rating.min())
print("Max item rating:", df.rating.max())
print("Mean item rating:", df.rating.mean())

Number of users: 69878
Number of items: 10677
Min item rating: 0.5
Max item rating: 5.0
Mean item rating: 3.512421932921562


## Load movie metadata
  - remove 10% as holdout test set

In [6]:
movies = pd.read_csv(path+'movies.tsv', sep='\t')
movies.head()

,movieId,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,Boomerang (1992),0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
1,1,"Net, The (1995)",1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
2,2,Dumb & Dumber (1994),0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,Outbreak (1995),1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0
4,4,Stargate (1994),1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [7]:
movies, movies_holdout = train_test_split(movies, test_size=.1, random_state=42)
movies.shape, movies_holdout.shape

((9609, 20), (1068, 20))

---
# Create baseline features
For each user, calculate average user bias - the average difference between the user's rating and the movie's average rating:

$$b_{u} = \dfrac{\sum_{j=1}^{n_u} (r_{uj} - \mu_i)}{n_u}$$

For each item, calculate the difference between its average rating and the average rating of all movies:

$$b_{i} = \dfrac{\sum_{k=1}^{n_i} (r_{ki})}{n_i} - \mu$$

Then, for each interaction, calculate the combined bias:

$$b_{ui} = \dfrac{b_u + b_i}{2}$$

In [8]:
def get_baseline(df, train_index, test_index):
    
    '''
    Calculate baseline features from an explicit ratings dataset. Receives a dataframe
    and returns train and test splits with added bias column and mean rating value.
    User and item biases are calculated as average difference from global mean rating.
    Baseline factors are only calculated from training observations, with users or
    items that do not appear in train receiving the global average as default.
    
    Args:
        df          : explicit ratings dataframe with columns userId, movieId and rating
        train_index : train index splits taken from KFold.splits()
        test_index  : test index splits taken from KFold.splits()
        
    Returns:
        train, test : train/test splits of df, with added bias column
        global_mean : average rating of all training observations
    '''
    
    train = df.iloc[train_index]
    test = df.iloc[test_index]
    
    # compute global mean
    global_mean = train.rating.mean()

    # compute average item ratings
    item_averages = train.groupby(
        'movieId'
    ).agg(
        {'rating':'mean'}
    ).rename(
        {'rating': 'item_avg'}, axis=1
    ).reset_index()
    
    # add as column to train and test
    train = pd.merge(train, item_averages, how='left', on='movieId')
    test = pd.merge(test, item_averages, how='left', on='movieId').fillna(global_mean)
    
    # compute average user bias
    train['user_bias'] = train['rating'] - train['item_avg']
    
    user_biases = train.groupby(
        'userId'
    ).agg(
        {'user_bias':'mean'}
    ).rename(
        {'user_bias': 'user_avg'}, axis=1
    ).reset_index()
    
    # add as column to train and test
    train = pd.merge(train, user_biases, how='left', on='userId')
    test = pd.merge(test, user_biases, how='left', on='userId').fillna(0.0)
    
    # interaction bias
    train['bias'] = (train['user_avg'] + train['item_avg'] - global_mean)/2
    test['bias'] = (test['user_avg'] + test['item_avg'] - global_mean)/2
    
    return train, test, global_mean

---
# Genre model
**To do**:
  - Can we avoid re-training rating model on CV fold?
  - Create a grid search function / class

In [9]:
def compile_genre_model(n_items, n_users, mean_rating, n_latent, n_hidden_1, n_hidden_2, dropout_1=.25, dropout_2=.25):
    
    # item latent factors
    item_in = Input(shape=[1])  # name='item'
    item_em = Embedding(n_items, n_latent)(item_in)
    item_vec = Flatten()(item_em)
    
    # user latent factors
    user_in = Input(shape=[1])
    user_em = Embedding(n_users, n_latent)(user_in)
    user_vec = Flatten()(user_em)
    
    # user x item bias
    bias = Input(shape=[1])
    
    # concatenate user and item vectors
    conc = Concatenate()([item_vec, user_vec])
    # hidden layer
    hidden_1 = Dense(n_hidden_1)(conc)
    leaky = LeakyReLU(alpha=.1)(hidden_1)
    drop_1 = Dropout(dropout_1)(leaky)
    
    out = Dense(1)(drop_1)
            
    rating = tf.math.add(Add()([out, bias]), mean_rating)
    
    # create model and compile it
    model = Model([user_in, item_in, bias], rating)
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    # model 2
    hidden_2 = Dense(n_hidden_2, activation='relu')(item_vec)
    drop_2 = Dropout(dropout_2)(hidden_2)
    genre = Dense(1, activation='sigmoid')(drop_2)

    # Create model and compile it
    model2 = Model(item_in, genre)
    # freeze the embedding layer
    model2.layers[1].trainable = False
    model2.compile(optimizer='adam', loss='binary_crossentropy' , metrics=['accuracy', 'AUC'])
    
    return model, model2

### Re-train model on full dataset, with best hparams

In [10]:
# get baseline predictors for full dataset
train, _, _ = get_baseline(df, df.index, df.index)

# compile both models
model1, model2 = compile_genre_model(
    n_items = df.movieId.nunique(),
    n_users = df.userId.nunique(),
    mean_rating = df.rating.mean(), 
    n_latent=80,
    n_hidden_1=75,
    n_hidden_2=75
)

# train rating model
ratings = model1.fit(
    x=[train.userId.values, train.movieId.values, train.bias.values],
    y=train.rating.values, 
    batch_size=1028,
    epochs=8,
    verbose=1,
    validation_split=.2
)

# train genre model
genres = model2.fit(
    movies.movieId.values, movies.Drama.values,
    batch_size=32, 
    epochs=7)

Train on 8000043 samples, validate on 2000011 samples
Epoch 1/8
8000043/8000043 [==============================] - 690s 86us/sample - loss: 0.7245 - val_loss: 0.6933
Epoch 2/8
8000043/8000043 [==============================] - 692s 87us/sample - loss: 0.6784 - val_loss: 0.6723
Epoch 3/8
8000043/8000043 [==============================] - 702s 88us/sample - loss: 0.6562 - val_loss: 0.6630
Epoch 4/8
8000043/8000043 [==============================] - 688s 86us/sample - loss: 0.6399 - val_loss: 0.6534
Epoch 5/8
8000043/8000043 [==============================] - 686s 86us/sample - loss: 0.6258 - val_loss: 0.6468
Epoch 6/8
8000043/8000043 [==============================] - 688s 86us/sample - loss: 0.6143 - val_loss: 0.6424
Epoch 7/8
8000043/8000043 [==============================] - 693s 87us/sample - loss: 0.6042 - val_loss: 0.6391
Epoch 8/8
8000043/8000043 [==============================] - 717s 90us/sample - loss: 0.5953 - val_loss: 0.6357
Train on 9609 samples
Epoch 1/7
9609/9609 [=======

### Evaluate on test set
**Best**: 70% with 80 : 75 | 75 hparams

In [11]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score

In [12]:
# class proportions
movies_holdout.Drama.value_counts()/len(movies_holdout)

0    0.515918
1    0.484082
Name: Drama, dtype: float64

In [13]:
X_test = movies_holdout.movieId.values
y_test = movies_holdout.Drama.values
y_score = pd.DataFrame(model2.predict(X_test))
y_pred = y_score.round().astype('int')

In [14]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.69      0.71       551
           1       0.69      0.73      0.71       517

    accuracy                           0.71      1068
   macro avg       0.71      0.71      0.71      1068
weighted avg       0.71      0.71      0.71      1068



In [15]:
pd.DataFrame(confusion_matrix(y_test, y_pred))

,0,1
0,378,173
1,140,377
